In [ ]:
# SETUP yours
user_libs = "C:/Users/User/AppData/Local/R/cache/R/renv/library/functional-data-regression-mip-e3349204/R-4.3/x86_64-w64-mingw32"
.libPaths(c(user_libs,.libPaths()))



In [ ]:


# run_simulation.R
# This script contains the function to run the simulation and returns all relevant outputs.

library(refund)
library(MASS)
library(fda)
library(here)

# Source the generic simulator script
source(here("src", "R", "generic_simulator", "simulate_main.R"))
source(here("src", "R", "generic_simulator", "utils","loader_utilities.R"))


# Define the function to run the simulation
run_simulation <- function(params) {
  # Ensure that the seed is set for reproducibility
  
  # Call the main analysis function with the parameters
  result <- do.call(generate_data, params[c("predictors", "observations", "measurements", "basis_functions", "intercept", "norder", "mu_funcs", "beta_funcs","time_domains", "cov_funcs", "seed","noise_snr","simulation_type")] )
    
  # Return the output list
  return(result)
}


simulation_name = "paper"
simulation_settings_file = "default"
# Required inputs before running the simulation!!!
inputs  <- load_simulation_settings(simulation_name, simulation_settings_file)
inputs$measurements <- 100
inputs$observations <- 400
inputs$basis_functions <- 6
time_domains_eval <- lapply(inputs$time_domains, function(domain) {
           seq(from = domain[[1]], to = domain[[2]], length.out = inputs$measurements)
       })
inputs$time_domains <- time_domains_eval
inputs$noise_snr <- c(FALSE,100)
outputs <- run_simulation(inputs);



[1] "predictors: 6"
[1] "observations: 400"
[1] "measurements: 100"
[1] "basis_functions: 6"
[1] "intercept: 0"


In [ ]:
library(here)
source(here( "simulations", "load_and_run_r_version.R"))


library(grplasso)
X = outputs$Z
Y = outputs$Y

[1] "predictors: 6"
[1] "observations: 400"
[1] "measurements: 100"
[1] "basis_functions: 6"
[1] "intercept: 0"


Warning message:
"il pacchetto 'grplasso' è stato creato con R versione 4.3.0"


In [ ]:
obseravation = dim(X)[1]    
predictors = dim(X)[2]
measurements = dim(X)[3]

X_matrix <- array(data = X, dim = c(obseravation, predictors * measurements))
group <- rep(1:predictors, each = measurements) # This is an example where each measurement for a predictor is considered its own group

dim(X_matrix)

[1] 400  36

In [ ]:
Y_vector <- as.numeric(Y)
index <- c( rep(1:predictors, each = measurements))


if(inputs$intercept > 0)
{
    index <- c(NA, rep(1:predictors, each = measurements))
    intercept_column <- rep(1, observations)
    X_matrix <- cbind(intercept_column, X_matrix)
}
index

[1] 1 1 1 1 1 1 2 2 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 4 5 5 5 5 5 5 6 6 6 6 6 6

In [ ]:
grpl = grplasso(x = X_matrix, y = Y_vector, index = index, lambda = 200,model = LinReg (),center=FALSE)

Lambda: 200  nr.var: 24 


In [ ]:
# Assuming your grplasso model is named 'grpl'

# Print the structure of the model object to understand what's available
print("Structure of the grplasso model object:")
str(grpl)

# Get and print the coefficients from the model
coefficients <- coef(grpl)
print("Coefficients from the grplasso model:")
print(coefficients)

# Identify and print non-zero coefficients to see which variables were selected
selected_variables <- which(coefficients != 0)
print("Indices of selected variables with non-zero coefficients:")
print(selected_variables)

# If the grplasso object contains fitted values, calculate and print R-squared
if ("fitted.values" %in% names(grpl)) {
  fitted_values <- grpl$fitted.values
  r_squared <- cor(Y, fitted_values)^2
  print("R-squared for the grplasso model:")
  print(r_squared)
}



[1] "Structure of the grplasso model object:"


List of 18
 $ x                : NULL
 $ y                : num [1:400] 6.4 8.93 6.39 7.03 9.08 ...
 $ coefficients     : num [1:36, 1] 0 0 0 0 0 0 0 0 0 0 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr "200"
 $ norms.pen        : num [1:6, 1] 0 0 1.26 3.05 2.17 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : NULL
  .. ..$ : chr "200"
 $ lambda           : num 200
 $ index            : int [1:36] 1 1 1 1 1 1 2 2 2 2 ...
 $ penscale         :function (x)  
 $ model            :Formal class 'grpl.model' [package "grplasso"] with 8 slots
  .. ..@ invlink  :function (eta)  
  .. ..@ link     :function (mu)  
  .. ..@ nloglik  :function (y, eta, weights, ...)  
  .. ..@ ngradient:function (x, y, mu, weights, ...)  
  .. ..@ nhessian :function (x, mu, weights, ...)  
  .. ..@ check    :function (y)  
  .. ..@ name     : chr "Linear Regression Model"
  .. ..@ comment  : chr "Use update.hess=\"lambda\" in grpl.control because the Hessian is constant"
 $ ngradient    

In [ ]:
compute_predictions_and_error <- function(coef_matrix, Z, Y_actual, true_predictors) {
  # Reshape coef_matrix to match the structure of Z
  reshaped_coefs <- as.vector(t(coef_matrix))
  
  # Compute predicted values
  Y_predicted <- Z %*% reshaped_coefs
  
  # Compute prediction error (MSE)
  mse <- mean((Y_actual - Y_predicted)^2)
  
  # Evaluate feature selection accuracy
  # Determine which predictors are selected by the model (non-zero coefficients)
  selected_predictors <- apply(coef_matrix, 1, function(row) any(row != 0))
  
  # Compare with true_predictors to calculate feature selection accuracy
  correct_selections <- sum(selected_predictors == true_predictors)
  total_predictors <- length(true_predictors)
  selection_accuracy <- correct_selections / total_predictors
  
  return(list(predicted = Y_predicted, mse = mse, accuracy = selection_accuracy))
}

to_matrix_form <-  function(predictor_coefficients, predictors, basis_functions) {
    # Create a matrix with 'basis_functions' elements per row
    # and the number of rows equal to the number of predictors from coeff_full
    coeff_matrix = matrix(0, nrow = predictors, ncol = basis_functions)

    for (i in 1:(predictors * basis_functions)) {
        # Calculate the row index as the ceiling of the division of 'i' by 'basis_functions'
        row_index = (i - 1) %/% basis_functions + 1
        # Calculate the column index as 'i' modulo 'basis_functions'; if modulo is 0, it means it's the last measurement
        col_index = ifelse(i %% basis_functions == 0, basis_functions, i %% basis_functions)
        # Assign the value of the coefficient to the matrix
        coeff_matrix[row_index, col_index] = predictor_coefficients[i]
}

  return(coeff_matrix)
}


true_predictors = inputs$true_predictors



In [ ]:
Y[1:10]

[1] 6.403068 8.926424 6.393052 7.028843 9.080664 5.886833 8.172387 5.557175
 [9] 9.518245 5.736347

In [ ]:
lambda = 10
grpl = grplasso(x = X_matrix, y = Y_vector, index = index, lambda = lambda,model = LinReg (),center=FALSE)

beta_star = to_matrix_form(grpl$coefficients, predictors, basis_functions)
beta_star

Lambda: 10  nr.var: 36 


1.569934e-02,1.457374e-02,2.000958e-03,3.370529e-02,-5.753362e-04,3.255108e-04
8.264664e-06,1.090621e-05,7.130755e-07,1.871545e-05,1.670581e-07,6.620571e-08
4.690114e-01,8.109023e-01,9.571828e-03,1.037594e+00,2.874453e-02,-5.704280e-04
4.737688e-01,8.700572e-01,2.536293e-01,7.832537e-01,-3.827963e-02,4.026181e-03
9.895640e-01,1.419081e+00,-5.943693e-02,1.202684e+00,3.752100e-02,-6.119584e-03
6.710540e-01,5.403322e-01,-7.882889e-02,6.931978e-01,1.406572e-01,-1.546046e-02


In [ ]:
beta_matrix = outputs$B
beta_matrix

5.012281e-06,0.1111006,0.3333244,0.6299252,0.7814756,0.8414660
1.764154e-04,0.2323649,0.6976152,1.0681552,0.9830768,0.8659877
0.000000e+00,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000
1.764154e-04,0.2323649,0.6976152,1.0681552,0.9830768,0.8659877
0.000000e+00,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000
0.000000e+00,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000


In [ ]:
max(beta_star)

[1] 1.296336

In [ ]:
beta_funcs = inputs$beta_funcs
time_domains = inputs$time_domains
Betas <- create_beta_curves(beta_funcs, time_domains)


In [ ]:
Betas

0,0.01010084,0.02020065,0.03029839,0.04039305,0.05048358,0.06056897,0.07064817,0.08072016,0.09078392,...,0.7889455,0.7951118,0.8011970,0.8072005,0.8131216,0.8189598,0.8247144,0.8303848,0.8359706,0.8414710
0,0.02115393,0.04229839,0.06342392,0.08452107,0.10558039,0.12659245,0.14754787,0.16843725,0.18925124,...,0.9450008,0.9378706,0.9303206,0.9223543,0.9139752,0.9051870,0.8959938,0.8863995,0.8764086,0.8660254
0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,...,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000
0,0.02115393,0.04229839,0.06342392,0.08452107,0.10558039,0.12659245,0.14754787,0.16843725,0.18925124,...,0.9450008,0.9378706,0.9303206,0.9223543,0.9139752,0.9051870,0.8959938,0.8863995,0.8764086,0.8660254
0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,...,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000
0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,...,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000


In [ ]:
outputs$beta_point_values

0,0.01010084,0.02020065,0.03029839,0.04039305,0.05048358,0.06056897,0.07064817,0.08072016,0.09078392,...,0.7889455,0.7951118,0.8011970,0.8072005,0.8131216,0.8189598,0.8247144,0.8303848,0.8359706,0.8414710
0,0.02115393,0.04229839,0.06342392,0.08452107,0.10558039,0.12659245,0.14754787,0.16843725,0.18925124,...,0.9450008,0.9378706,0.9303206,0.9223543,0.9139752,0.9051870,0.8959938,0.8863995,0.8764086,0.8660254
0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,...,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000
0,0.02115393,0.04229839,0.06342392,0.08452107,0.10558039,0.12659245,0.14754787,0.16843725,0.18925124,...,0.9450008,0.9378706,0.9303206,0.9223543,0.9139752,0.9051870,0.8959938,0.8863995,0.8764086,0.8660254
0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,...,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000
0,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,...,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000,0.0000000


In [ ]:

 
 # Call the appropriate data generation function based on simulation type
  if (simulation_type == "paper") {
    data <- simulate_paper_data(mu_funcs, beta_funcs, observations, time_domains, intercept, predictors, noise_snr)
  } else{
    data <- simulate_cov_data(mu_funcs, cov_funcs, beta_funcs, observations, time_domains, intercept, predictors, noise_snr) 
  }

  # Extract U, X, Y from the returned data
  U <- data$U
  X <- data$X
  Y <- data$Y

  # Remaining processing steps
  basis_objs <- create_basis(basis_functions, time_domains, norder, predictors)
  result <- smooth_betas_generic(beta_funcs, basis_functions, time_domains, basis_objs)
  Beta_matrix <- result$beta_matrix
  basis_values <- result$basis_values
  beta_point_values <- result$beta_point_values
  J <- compute_J_matrix_generic(basis_objs, predictors, basis_functions)
  W <- compute_W_matrix_generic(X, basis_functions, time_domains, basis_objs)
  Z_matrix <- compute_Z_matrix_generic(W, J, predictors, basis_functions)

In [ ]:
err = 0 
for (K in 1:100){
    cumulative = 0 
for (i in 1:predictors) {
    cumulative = cumulative +t(W[K,i,]) %*% J[i,,] %*% Beta_matrix[i,]
}
# reconstruct Y[1] from W J and beta
cumulative2 = 0 
for (i in 1:predictors) {
    cumulative2 = cumulative + t(Z_matrix[K,i,]) %*% Beta_matrix[i,]
}



err = err + (Y[K] - cumulative2 )
}
# reconstruct Y[1] from W J and beta
print(err)

          [,1]
[1,] -2.101535
